In [ ]:
"""
Standard MPC for Passing through a dynamic gate
"""
import casadi as ca
import numpy as np
import time
from os import system
#
class MPC(object):
    """
    Nonlinear MPC
    """
    def __init__(self, T, dt, so_path='./nmpc.so'):
        """
        Nonlinear MPC for quadrotor control        
        """
        self.so_path = so_path

        # Time constant
        self._T = T
        self._dt = dt
        self._N = int(self._T/self._dt)

        # Gravity
        self._gz = 9.81

        # bluerov parameters
        self._w_max_yaw = 6.0
        self._w_max_xy = 6.0
        self._thrust_min = 2.0
        self._thrust_max = 20.0
        
        self.x_g = 0
        self.y_g = 0
        self.z_g = 0
        self.x_b = 0
        self.y_b = 0
        self.z_b = -0.01

        #added mass coefficients
        self.X_ud = 6.356673886738176
        self.Y_vd = 7.120600295756984
        self.Z_wd = 18.686326861534997
        self.K_pd = 0.185765630747592
        self.M_qd = 0.134823349429660
        self.N_rd = 0.221510466644690
        #drag 
        self.Xu = {"Linear": 13.7, "NonLinear": 141}
        self.Yv = {"Linear": 0, "NonLinear": 217}
        self.Zw = {"Linear": 33, "NonLinear": 190}
        self.Kp = {"Linear": 0, "NonLinear": 1.192}
        self.Mq = {"Linear": 0.8, "NonLinear": 0.470}
        self.Nr = {"Linear": 0, "NonLinear": 1.5}
        self.m = 13.5 # Mass of the Robot

        self.I_x, self.I_y, self.I_z = 0.26, 0.23, 0.37 # Mass Moments of Inertia

        self.L_h, self.L_w, self.L_l = 0.378, 0.575, 0.457 

        self.A_F, self.A_S, self.A_T = 0.0877, 0.1131, 0.2049

        self.Volume = 0.0135

        self.Ic = np.array([[0.26, 0, 0],
            [0, 0.23, 0],
            [0, 0, 0.37]])

        self.Ap_F, self.Ap_S, self.Ap_T = 0.1727, 0.2174, 0.2628
        #
        # state dimension (px, py, pz,           # bluerov position
        #                  qw, qx, qy, qz,       # bluerov quaternion
        #                  u, v, w,           # bluerov linear velocity
        #                  p, q, r)           # bluerov angular velocity
        self._s_dim = 12
        # action dimensions (c_thrust, wx, wy, wz)
        self._u_dim = 6
        
        # cost matrix for tracking the goal point
        self._Q_goal = np.diag([
            100, 100, 100,  # delta_x, delta_y, delta_z
            10, 10, 10, # delta_qw, delta_qx, delta_qy, delta_qz
            10, 10, 10,
            10, 10, 10]) 

        # cost matrix for the action
        self._Q_u = np.diag([0.1 for _ in range(self._u_dim)]) # T, wx, wy, wz

        # initial state and control action
        self._bluerov_s0 = [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
        self._bluerov_u0 = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

        self._initDynamics()
    


    def _initDynamics(self,):
        # # # # # # # # # # # # # # # # # # # 
        # ---------- Input States -----------
        # # # # # # # # # # # # # # # # # # # 

        px, py, pz = ca.SX.sym('px'), ca.SX.sym('py'), ca.SX.sym('pz')
        #
        phi, theta, psi = ca.SX.sym('phi'), ca.SX.sym('theta'), ca.SX.sym('psi')
        #
        u, v, w = ca.SX.sym('u'), ca.SX.sym('v'), ca.SX.sym('w')
        p, q, r = ca.SX.sym('p'), ca.SX.sym('q'), ca.SX.sym('r')

        # -- conctenated vector
        self._x = ca.vertcat(px, py, pz, phi, theta, psi, u, v, w, p, q, r) 

        # # # # # # # # # # # # # # # # # # # 
        # --------- Control Command ------------
        # # # # # # # # # # # # # # # # # # #

        X, Y, Z, K, M, N = ca.SX.sym('X'), ca.SX.sym('Y'), ca.SX.sym('Z'),\
            ca.SX.sym('K'), ca.SX.sym('M'), ca.SX.sym('N')
        
        # -- conctenated vector
        self._u = ca.vertcat(X, Y, Z, K, M, N)
        
        # # # # # # # # # # # # # # # # # # # 
        # --------- System Dynamics ---------
        # # # # # # # # # # # # # # # # # # #
        # # # # # # # # Mass Matrix # # # # # # # # # # #
        #Rigid-body mass
        M_RB = np.array([
            [self.m, 0, 0, 0, 0, 0],
            [0, self.m, 0, 0, 0, 0],
            [0, 0, self.m, 0, 0, 0],
            [0, 0, 0, self.I_x, 0, 0 ],
            [0, 0, 0, 0, self.I_y, 0 ],
            [0, 0, 0, 0, 0, self.I_z ]
            ])
        #added mass
        M_A = -1 * np.array([[self.X_ud, 0, 0, 0, 0, 0],
                    [0, self.Y_vd, 0, 0, 0, 0],
                    [0, 0, self.Z_wd, 0, 0, 0],
                    [0, 0, 0, self.K_pd, 0, 0 ],
                    [0, 0, 0, 0, self.M_qd, 0 ],
                    [0, 0, 0, 0, 0, self.N_rd ]])
        M = M_RB + M_A
        # # # # # # # # # # Coriolis # # # # # # # # #
        C_RB = np.array([
            [0, 0, 0, 0, self.m*w, -self.m*v ],
            [0, 0, 0, -self.m*w, 0, self.m*u ],
            [0, 0, 0, self.m*v, -self.m*u, 0 ],
            [0, self.m*w, -self.m*v, 0, -self.I_z*r, -self.I_y*q ],
            [-self.m*w, 0, self.m*u, self.I_z*r, 0, self.I_x*p ],
            [self.m*v, -self.m*u, 0, self.I_y*q, -self.I_x*p, 0 ],
            ])

        # added mass Coriolis
        C_A = np.array([
            [0, 0, 0, 0, -self.Z_wd*w, self.Y_vd*v],
            [0, 0, 0, self.Z_wd*w, 0,-self.X_ud*u], 
            [0, 0, 0, -self.Y_vd*v, self.X_ud*u, 0],
            [0, -self.Z_wd*w, self.Y_vd*v, 0, -self.N_rd*r, self.M_qd*q],
            [self.Z_wd*w, 0, -self.X_ud*u, self.N_rd*r, 0, -self.K_pd*p],
            [-self.Y_vd*v, self.X_ud*u, 0, -self.M_qd*q, self.K_pd*p, 0],
            ])

        C = C_RB + C_A
        # # # # # # # # # # Drag # # # # # # # # #
        D_lin = np.array([
            [self.Xu["Linear"], 0, 0, 0, 0, 0],
            [0, self.Yv["Linear"], 0, 0, 0, 0],
            [0, 0, self.Zw["Linear"], 0, 0, 0],
            [0, 0, 0, self.Kp["Linear"], 0, 0],
            [0, 0, 0, 0, self.Mq["Linear"], 0],
            [0, 0, 0, 0, 0, self.Nr["Linear"]]
        ])
            
        D_nonlin = np.array([
            [self.Xu["NonLinear"] * u, 0, 0, 0, 0, 0 ],
            [0, self.Yv["NonLinear"] * v, 0, 0, 0, 0 ],
            [0, 0, self.Zw["NonLinear"] * w, 0, 0, 0 ],
            [0, 0, 0, self.Kp["NonLinear"] * p, 0, 0 ],
            [0, 0, 0, 0, self.Mq["NonLinear"] * q, 0 ],
            [0, 0, 0, 0, 0, self.Nr["NonLinear"] * r ],        
        ])

        D = D_lin + D_nonlin
        G = ca.vertcat(
            (W - B) * sin(theta),
            -(W - B) * cos(theta) * sin(phi),
            -(W - B) * cos(theta) * cos(phi),
            -(y_g*W - y_b*B)*cos(theta)*cos(phi) + (z_g*W - z_b*B)*cos(theta)*sin(phi),
            (z_g*W - z_b*B)*sin(theta) + (x_g*W - x_b*B)*cos(theta)*cos(phi),
            -(x_g*W - x_b*B)*cos(theta)*sin(phi) - (y_g*W - y_b*B)*sin(theta)
            )
        
        v_dot =  ca.inv( M ) @ (self._u  - C @ self._x[5:] - D @ self._x[5:] - G )
        
        x_dot = ca.vertcat(
            vx,
            vy,
            vz,
            0.5 * ( -wx*qx - wy*qy - wz*qz ),
            0.5 * (  wx*qw + wz*qy - wy*qz ),
            0.5 * (  wy*qw - wz*qx + wx*qz ),
            0.5 * (  wz*qw + wy*qx - wx*qy ),
            2 * ( qw*qy + qx*qz ) * thrust,
            2 * ( qy*qz - qw*qx ) * thrust, 
            (qw*qw - qx*qx -qy*qy + qz*qz) * thrust - self._gz
            # (1 - 2*qx*qx - 2*qy*qy) * thrust - self._gz
        )

        #
        self.f = ca.Function('f', [self._x, self._u], [x_dot], ['x', 'u'], ['ode'])
                
        # # Fold
        F = self.sys_dynamics(self._dt)
        fMap = F.map(self._N, "openmp") # parallel
        
        # # # # # # # # # # # # # # # 
        # ---- loss function --------
        # # # # # # # # # # # # # # # 

        # placeholder for the quadratic cost function
        Delta_s = ca.SX.sym("Delta_s", self._s_dim)
        Delta_p = ca.SX.sym("Delta_p", self._s_dim)
        Delta_u = ca.SX.sym("Delta_u", self._u_dim)        
        
        #        
        cost_goal = Delta_s.T @ self._Q_goal @ Delta_s 
        cost_gap = Delta_p.T @ self._Q_pen @ Delta_p 
        cost_u = Delta_u.T @ self._Q_u @ Delta_u

        #
        f_cost_goal = ca.Function('cost_goal', [Delta_s], [cost_goal])
        f_cost_gap = ca.Function('cost_gap', [Delta_p], [cost_gap])
        f_cost_u = ca.Function('cost_u', [Delta_u], [cost_u])

        #
        # # # # # # # # # # # # # # # # # # # # 
        # # ---- Non-linear Optimization -----
        # # # # # # # # # # # # # # # # # # # #
        self.nlp_w = []       # nlp variables
        self.nlp_w0 = []      # initial guess of nlp variables
        self.lbw = []         # lower bound of the variables, lbw <= nlp_x
        self.ubw = []         # upper bound of the variables, nlp_x <= ubw
        #
        self.mpc_obj = 0      # objective 
        self.nlp_g = []       # constraint functions
        self.lbg = []         # lower bound of constrait functions, lbg < g
        self.ubg = []         # upper bound of constrait functions, g < ubg

        u_min = [self._thrust_min, -self._w_max_xy, -self._w_max_xy, -self._w_max_yaw]
        u_max = [self._thrust_max,  self._w_max_xy,  self._w_max_xy,  self._w_max_yaw]
        x_bound = ca.inf
        x_min = [-x_bound for _ in range(self._s_dim)]
        x_max = [+x_bound for _ in range(self._s_dim)]
        #
        g_min = [0 for _ in range(self._s_dim)]
        g_max = [0 for _ in range(self._s_dim)]

        P = ca.SX.sym("P", self._s_dim+(self._s_dim+3)*self._N+self._s_dim)
        X = ca.SX.sym("X", self._s_dim, self._N+1)
        U = ca.SX.sym("U", self._u_dim, self._N)
        #
        X_next = fMap(X[:, :self._N], U)
        
        # "Lift" initial conditions
        self.nlp_w += [X[:, 0]]
        self.nlp_w0 += self._quad_s0
        self.lbw += x_min
        self.ubw += x_max
        
        # # starting point.
        self.nlp_g += [ X[:, 0] - P[0:self._s_dim]]
        self.lbg += g_min
        self.ubg += g_max
        
        for k in range(self._N):
            #
            self.nlp_w += [U[:, k]]
            self.nlp_w0 += self._bluerov_u0
            self.lbw += u_min
            self.ubw += u_max
            
            # retrieve time constant
            # idx_k = self._s_dim+self._s_dim+(self._s_dim+3)*(k)
            # idx_k_end = self._s_dim+(self._s_dim+3)*(k+1)
            # time_k = P[ idx_k : idx_k_end]

            # cost for tracking the goal position
            cost_goal_k, cost_gap_k = 0, 0
            if k >= self._N-1: # The goal postion.
                delta_s_k = (X[:, k+1] - P[self._s_dim+(self._s_dim+3)*self._N:])
                cost_goal_k = f_cost_goal(delta_s_k)
            else:
                # cost for tracking the moving gap
                delta_p_k = (X[:, k+1] - P[self._s_dim+(self._s_dim+3)*k : \
                    self._s_dim+(self._s_dim+3)*(k+1)-3]) 
                cost_gap_k = f_cost_gap(delta_p_k)
        
            delta_u_k = U[:, k]-[self._gz, 0, 0, 0]
            cost_u_k = f_cost_u(delta_u_k)

            self.mpc_obj = self.mpc_obj + cost_goal_k + cost_u_k +  cost_gap_k 

            # New NLP variable for state at end of interval
            self.nlp_w += [X[:, k+1]]
            self.nlp_w0 += self._bluerov_s0
            self.lbw += x_min
            self.ubw += x_max

            # Add equality constraint
            self.nlp_g += [X_next[:, k] - X[:, k+1]]
            self.lbg += g_min
            self.ubg += g_max

        # nlp objective
        nlp_dict = {'f': self.mpc_obj, 
            'x': ca.vertcat(*self.nlp_w), 
            'p': P,               
            'g': ca.vertcat(*self.nlp_g) }        
        
        # # # # # # # # # # # # # # # # # # # 
        # -- qpoases            
        # # # # # # # # # # # # # # # # # # # 
        # nlp_options ={
        #     'verbose': False, \
        #     "qpsol": "qpoases", \
        #     "hessian_approximation": "gauss-newton", \
        #     "max_iter": 100, 
        #     "tol_du": 1e-2,
        #     "tol_pr": 1e-2,
        #     "qpsol_options": {"sparse":True, "hessian_type": "posdef", "numRefinementSteps":1} 
        # }
        # self.solver = ca.nlpsol("solver", "sqpmethod", nlp_dict, nlp_options)
        # cname = self.solver.generate_dependencies("mpc_v1.c")  
        # system('gcc -fPIC -shared ' + cname + ' -o ' + self.so_path)
        # self.solver = ca.nlpsol("solver", "sqpmethod", self.so_path, nlp_options)
        

        # # # # # # # # # # # # # # # # # # # 
        # -- ipopt
        # # # # # # # # # # # # # # # # # # # 
        ipopt_options = {
            'verbose': False, \
            "ipopt.tol": 1e-4,
            "ipopt.acceptable_tol": 1e-4,
            "ipopt.max_iter": 100,
            "ipopt.warm_start_init_point": "yes",
            "ipopt.print_level": 0, 
            "print_time": False
        }
        
        # self.solver = ca.nlpsol("solver", "ipopt", nlp_dict, ipopt_options)
        # # jit (just-in-time compilation)
        # print("Generating shared library........")
        # cname = self.solver.generate_dependencies("mpc_v1.c")  
        # system('gcc -fPIC -shared -O3 ' + cname + ' -o ' + self.so_path) # -O3
        
        # # reload compiled mpc
        print(self.so_path)
        self.solver = ca.nlpsol("solver", "ipopt", self.so_path, ipopt_options)

    def solve(self, ref_states):
        # # # # # # # # # # # # # # # #
        # -------- solve NLP ---------
        # # # # # # # # # # # # # # # #
        #
        self.sol = self.solver(
            x0=self.nlp_w0, 
            lbx=self.lbw, 
            ubx=self.ubw, 
            p=ref_states, 
            lbg=self.lbg, 
            ubg=self.ubg)
        #
        sol_x0 = self.sol['x'].full()
        opt_u = sol_x0[self._s_dim:self._s_dim+self._u_dim]

        # Warm initialization
        self.nlp_w0 = list(sol_x0[self._s_dim+self._u_dim:2*(self._s_dim+self._u_dim)]) + list(sol_x0[self._s_dim+self._u_dim:])
        
        #
        x0_array = np.reshape(sol_x0[:-self._s_dim], newshape=(-1, self._s_dim+self._u_dim))
        
        # return optimal action, and a sequence of predicted optimal trajectory.  
        return opt_u, x0_array
    
    def sys_dynamics(self, dt):
        M = 4       # refinement
        DT = dt/M
        X0 = ca.SX.sym("X", self._s_dim)
        U = ca.SX.sym("U", self._u_dim)
        # #
        X = X0
        for _ in range(M):
            # --------- RK4------------
            k1 =DT*self.f(X, U)
            k2 =DT*self.f(X+0.5*k1, U)
            k3 =DT*self.f(X+0.5*k2, U)
            k4 =DT*self.f(X+k3, U)
            #
            X = X + (k1 + 2*k2 + 2*k3 + k4)/6        
        # Fold
        F = ca.Function('F', [X0, U], [X])
        return F
            

In [ ]:
import os
so_path = "./saved"
os.listdir(so_path)